In [9]:
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
import open3d as o3d

import os
import random

from pathlib import Path
from transformers import AutoImageProcessor, AutoModelForDepthEstimation

print(f"CUDA: {torch.cuda.is_available()}")

print("ROCm verfügbar:", torch.backends.mps.is_available())

# Anzahl der verfügbaren GPUs auf ROCm
print("Anzahl ROCm-GPUs:", torch.cuda.device_count())

# Name der ersten GPU
if torch.cuda.is_available():
    print("GPU-Name:", torch.cuda.get_device_name(0))
else:
    print("Keine GPU verfügbar.")

RuntimeError: Failed to import transformers.models.auto.image_processing_auto because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [ ]:
# Randomly sample images from the indoor and outdoor datasets

# Read in sample images
data_folder_path = Path("data")
indoor_folder_path = data_folder_path / "indoor"
outdoor_folder_path = data_folder_path / "outdoor"

num_of_samples = 3

selected_indoor_images = random.sample(os.listdir(indoor_folder_path), num_of_samples)
selected_outdoor_images = random.sample(os.listdir(outdoor_folder_path), num_of_samples)

indoor_images = []
outdoor_images = []

for i in range(num_of_samples):
    indoor_image = cv2.imread(str(indoor_folder_path / selected_indoor_images[i]))
    indoor_image = cv2.cvtColor(indoor_image, cv2.COLOR_BGR2RGB)

    outdoor_image = cv2.imread(str(outdoor_folder_path / selected_outdoor_images[i]))
    outdoor_image = cv2.cvtColor(outdoor_image, cv2.COLOR_BGR2RGB)

    indoor_images.append(indoor_image)
    outdoor_images.append(outdoor_images)

    print(indoor_image.shape, outdoor_image.shape)


(431, 762, 3) (407, 600, 3)
(306, 400, 3) (407, 600, 3)
(480, 640, 3) (600, 407, 3)


https://github.com/LiheYoung/Depth-Anything

In [ ]:
# Import depth anything model smal, base and large

processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-large-hf")
model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-large-hf").to("cuda")

NameError: name 'AutoImageProcessor' is not defined

In [ ]:
# Pre process images

indoor_samples = []
outdoor_samples = []

for i in range(num_of_samples):
    indoor_input = processor(images=indoor_images[i], return_tensors="pt").to("cuda")
    outdoor_input = processor(images=outdoor_images[i], return_tensors="pt").to("cuda")

    # Infer model
    with torch.no_grad():
        indoor_outputs = model(**indoor_input)
        outdoor_outputs = model(**outdoor_input)

        indoor_depth = indoor_outputs.predicted_depth
        output_depth = outdoor_outputs.predicted_depth

    indoor_depth = indoor_depth.squezze().cpu().numpy()
    outdoor_depth = outdoor_depth.squezze().cpu().numpy()

    indoor_samples.append([indoor_images[i], indoor_depth])
    outdoor_samples.append([outdoor_images[i], outdoor_depth])

In [ ]:
for i in range(num_of_samples):
    # Visualize the depth image and the original image side by side alternate between indoor and outdoor